In [22]:
import os
os.chdir('D://scratch_backup//alpine//beri9208//lobster_test/diamond/diamond.save')
import numpy as np
import h5py
from scipy.io import FortranFile

In [47]:
# SAMPLE READ EXAMPLE FROM QE OUTPUT

with open('wfc1.dat', 'rb') as f:
    # Moves the cursor 4 bytes to the right
    # Fortran unformatted files will always output data with a 4 byte prefix and 4 byte suffix
    f.seek(4)

    ik = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    xk = np.fromfile(f, dtype='float64', count=3)  # FORTRAN REAL(8)
    ispin = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    gamma_only = np.fromfile(f, dtype='int32', count=1) # FORTRAN LOGICAL
    scalef = np.fromfile(f, dtype='float64', count=1)  # FORTRAN REAL(8)

    # Move the cursor 8 byte to the right
    f.seek(8, 1)

    ngw = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    igwx = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    npol = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    nbnd = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER

    # Move the cursor 8 byte to the right
    f.seek(8, 1)

    b1 = np.fromfile(f, dtype='float64', count=3)
    b2 = np.fromfile(f, dtype='float64', count=3)
    b3 = np.fromfile(f, dtype='float64', count=3)

    f.seek(8,1)

    mill = np.fromfile(f, dtype='int32', count=3*igwx[0])
    mill2 = mill.reshape( (igwx[0], 3) )

    evc = np.zeros( (nbnd[0], npol[0]*igwx[0]), dtype="complex128")

    f.seek(8,1)
    for i in range(nbnd[0]):
        evc[i,:] = np.fromfile(f, dtype='complex128', count=npol[0]*igwx[0])
        f.seek(8, 1)
write_vars = [ik, xk, ispin, gamma_only, scalef, ngw, igwx, npol, nbnd, b1, b2, b3, mill, mill2, evc]
write_vars_strings = ['ik', 'xk', 'ispin', 'gamma_only', 'scalef', 'ngw', 'igwx', 'npol', 'nbnd', 'b1', 'b2', 'b3', 'mill', 'mill2', 'evc']

In [154]:
#%reset
# WRITE A DAT FILE
import os
import numpy as np
from scipy.io import FortranFile
filename = "wfc_sample"
if os.path.exists("wfc_sample.dat"):
    os.remove("wfc_sample.dat")

# define the variables
ik = 2
xk = np.array([0.0, 0.0, 0.0], dtype=np.float64)
ispin = 2
gamma_only = 0
scalef = 1.0
ngw = 100
igwx = 50
npol = 2
nbnd = 10
b1 = np.array([1.0, 0.0, 0.0], dtype=np.float64)
b2 = np.array([0.0, 1.0, 0.0], dtype=np.float64)
b3 = np.array([0.0, 0.0, 1.0], dtype=np.float64)
mills = np.zeros((3, igwx), dtype=np.int32)
wtmp = np.ones((nbnd, npol*igwx), dtype=np.complex128)
# wtmp[0][0] = 1. + 0.j
# wtmp[1][0] = 1. + 0.j

packed_data = np.array([ik], dtype=np.int32).tobytes()
packed_data += xk.tobytes()
packed_data += np.array([ispin, gamma_only], dtype=np.int32).tobytes()
packed_data += np.array([scalef], dtype=np.float64).tobytes()

packed_data2 = np.concatenate((b1, b2, b3)).tobytes()

# write the variables to the file
with FortranFile(filename+'.dat', 'w') as f:
#    f.write_record(np.array([packed_size], dtype=np.int32))
    f.write_record(np.frombuffer(packed_data, dtype=np.int32))
#    f.write_record(padding_bytes)
    f.write_record(np.array([ngw, igwx, npol, nbnd], dtype=np.int32))
    # f.write_record(np.array([packed_size2], dtype=np.int32))
    f.write_record(np.frombuffer(packed_data2, dtype=np.uint8))
    # f.write_record(padding_bytes2)
    # f.write_record(np.frombuffer(packed_data2, dtype=np.uint8))
    f.write_record(mills)
    for j in range(nbnd):
        f.write_record(wtmp[j])
f.close()

In [155]:
# READ WRITTEN DAT

with open('wfc_sample.dat', 'rb') as f:
    # Moves the cursor 4 bytes to the right
    # Fortran unformatted files will always output data with a 4 byte prefix and 4 byte suffix
    f.seek(4)

    ik = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    print('ik: ' + str(ik))
    xk = np.fromfile(f, dtype='float64', count=3)  # FORTRAN REAL(8)
    print('xk: ' + str(xk))
    ispin = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    print('ispin: ' + str(ispin))
    gamma_only = np.fromfile(f, dtype='int32', count=1) # FORTRAN LOGICAL
    print('gamma_only: ' + str(gamma_only))
    scalef = np.fromfile(f, dtype='float64', count=1)  # FORTRAN REAL(8)
    print('scalef: ' + str(scalef))

    # Move the cursor 8 byte to the right
    f.seek(8, 1)

    ngw = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    print('ngw: ' + str(ngw))
    igwx = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    print('igwx: ' + str(igwx))
    npol = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    print('npol: ' + str(npol))
    nbnd = np.fromfile(f, dtype='int32', count=1) # FORTRAN INTEGER
    print('nbnd: ' + str(nbnd))

    # Move the cursor 8 byte to the right
    f.seek(8, 1)

    b1 = np.fromfile(f, dtype='float64', count=3)
    print('b1: ' + str(b1))
    b2 = np.fromfile(f, dtype='float64', count=3)
    print('b2: ' + str(b2))
    b3 = np.fromfile(f, dtype='float64', count=3)
    print('b3: ' + str(b3))

    f.seek(8,1)

    mill = np.fromfile(f, dtype='int32', count=3*igwx[0])
    print('mill: ' + str(mill))
    print(np.shape(mill))
    mill = mill.reshape( (igwx[0], 3) )

    evc = np.zeros( (nbnd[0], npol[0]*igwx[0]), dtype="complex128")

    f.seek(8,1)
    for i in range(nbnd[0]):
        use = np.fromfile(f, dtype='complex128', count=npol[0]*igwx[0])
        print(i)
        print(use[0])
        evc[i,:] = use
        f.seek(8, 1)
f.close()

ik: [2]
xk: [0. 0. 0.]
ispin: [2]
gamma_only: [0]
scalef: [1.]
ngw: [100]
igwx: [50]
npol: [2]
nbnd: [10]
b1: [1. 0. 0.]
b2: [0. 1. 0.]
b3: [0. 0. 1.]
mill: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
(150,)
0
(1+0j)
1
(1+0j)
2
(1+0j)
3
(1+0j)
4
(1+0j)
5
(1+0j)
6
(1+0j)
7
(1+0j)
8
(1+0j)
9
(1+0j)
